In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam

from keras.callbacks import EarlyStopping


In [26]:
X_train_all = []
y_train_all = []

train_files = ['/kaggle/input/test-anomaly/0.csv',
               '/kaggle/input/test-anomaly//2.csv',
               '/kaggle/input/test-anomaly/3.csv',
               '/kaggle/input/test-anomaly/4.csv',
               '/kaggle/input/test-anomaly/5.csv',
               
               '/kaggle/input/train-normal/0.csv',
               '/kaggle/input/train-normal/1.csv',
               '/kaggle/input/train-normal/2.csv',
               '/kaggle/input/train-normal/3.csv',
               '/kaggle/input/train-normal/4.csv',
               '/kaggle/input/train-normal/5.csv',
               '/kaggle/input/train-normal/6.csv',
               '/kaggle/input/train-normal/7.csv',
               '/kaggle/input/train-normal/8.csv']
for file in train_files:
    train_df = pd.read_csv(file)
    label_encoder = LabelEncoder()
    train_df['type_encoded'] = label_encoder.fit_transform(train_df['type'])
    X_train_all.append(train_df[['time_pos', 'type_encoded', 'byte_pos', 'size']].values)
    y_train_all.append(train_df[['is_normal', 'has_move', 'has_blur', 'has_block', 'has_overexposure']].values)

X_train = np.concatenate(X_train_all)
y_train = np.concatenate(y_train_all)


In [27]:
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax')) 

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [29]:
test_files = ['/kaggle/input/meta-test/1.mp4.csv',
              '/kaggle/input/meta-test/2.mp4.csv',
              '/kaggle/input/meta-test/3.mp4.csv',
              '/kaggle/input/meta-test/4.mp4.csv',
              '/kaggle/input/meta-test/5.mp4.csv'] 
for file in test_files:
    test_df = pd.read_csv(file)
    test_df['type_encoded'] = label_encoder.transform(test_df['type'])
    X_test = test_df[['time_pos', 'type_encoded', 'byte_pos', 'size']].values
    y_test = test_df[['is_normal', 'has_move', 'has_blur', 'has_block', 'has_overexposure']].values
    
    X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print("Test Accuracy:", accuracy)

Epoch 1/30
4936/4936 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.8693 - loss: 0.5352 - val_accuracy: 0.5838 - val_loss: 1.4135
Epoch 2/30
4936/4936 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.8776 - loss: 0.4993 - val_accuracy: 0.5838 - val_loss: 1.5041
Epoch 3/30
4936/4936 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.8789 - loss: 0.5014 - val_accuracy: 0.5838 - val_loss: 1.4873
Epoch 4/30
4936/4936 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.8775 - loss: 0.5028 - val_accuracy: 0.5838 - val_loss: 1.4161
Epoch 1/30
4936/4936 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.8780 - loss: 0.5035 - val_accuracy: 0.8667 - val_loss: 1.2256
Epoch 2/30
4936/4936 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.8789 - loss: 0.4880 - val_accuracy: 0.8667 - val_loss: 1.2667
Epoch 3/30
4936/4936 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.8784 - loss: 0.4996 - val_accuracy: 0.8667 - val_loss: 1.2670
Epoch 4/30
4936/4936 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.8783 - loss: 0

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None